# Задание 1 (10 баллов)

Обучите языковую модель с помощью GRPO на вот этом датасете - `notbadai/python_functions_reasoning`
Он чем-то похож на датасет из семинара, но вместо математических задач нужно написать код на питоне.
Измените system prompt под эту задачу. Напишите аналогичные reward функции - одна на проверку формата (markdown wrappers ```python... ``` вокруг кода в конце), и на проверку что код работает (можете использовать import ast; ast.parse(string) для проверки)
Если модель сначала не может отвечать в нужно формате, то сделайте цикл SFT обучения на небольшом куске датасета.
GRPO в колабе может работать долго, поэтому не старайтесь пропустить весь датасет. Но обучайте хотя бы на 100 промптах (по несколько генераций на каждый промпт).
Оцените разницу в качестве на небольшом сабсете, прогнав изначальную модель, модель после SFT (если она есть) и GRPO модель. В качестве метрики используйте reward функции.
Можете взять любую языковую модель любого размера. Можете дообучать всю модель или же только адаптера поверх квантизированной модели.


In [1]:
#!pip install -U bitsandbytes

In [2]:
#pip install datasets trl

In [3]:
import ast
from datasets import load_dataset
from trl import GRPOConfig, GRPOTrainer


from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from trl import DPOTrainer
from datasets import load_dataset
from tqdm.notebook import tqdm
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
import numpy as np

Exception ignored in: <function _xla_gc_callback at 0x79660ab1cd60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


In [5]:
model_name = "Qwen/Qwen2-0.5B-Instruct"
device = 'cuda'

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_4bit=True
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [6]:
model = model.to(device)

In [7]:
dataset = load_dataset("notbadai/python_functions_reasoning", "default")['train']
dataset_train = dataset.select([i for i in range(100)])
dataset_test = dataset.select([i for i in range(500, 700)])

In [8]:
system_prompt = """
Your task is to write Python code based on the user task. Your code must be valid and wrapped into ```python *your code*```
"""

def chatml_format(example):
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": example['prompt']}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    return {
        "prompt": prompt
    }



In [17]:
import re

def format_accuracy_func(completions, **kwargs):
    rewards = []
    for response in completions:
        if re.search('```python\n(.*?)\n```', response, re.DOTALL):
            rewards.append(1.0)

        else:
            rewards.append(0.0)

    return rewards

def answer_accuracy_func(completions, **kwargs):
    rewards = []
    for response in completions:
        try:
            match_c = re.search('```python\n(.*?)\n```', response, re.DOTALL)
            ast.parse(match_c.group(1))
            rewards.append(1.0)
        except Exception:
            rewards.append(0.0)

    return rewards



In [10]:
dataset_train = dataset_train.map(chatml_format)
dataset_test = dataset_test.map(chatml_format)

In [11]:
batch = dataset_train[2:3]

In [12]:
input_ids = tokenizer.batch_encode_plus(batch['prompt'], return_tensors='pt', padding=True)

output = model.generate(
    input_ids['input_ids'].to(device), attention_mask=input_ids['attention_mask'].to(device),
    max_new_tokens=400, do_sample=True, temperature=1.5, pad_token_id=tokenizer.eos_token_id
)

/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


In [13]:
completions = tokenizer.batch_decode(output, skip_special_tokens=False)

In [18]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules="all-linear"
)

In [19]:
training_args = GRPOConfig(output_dir="Qwen/Qwen2-0.5B-Instruct-python-sft-grpo",
                           logging_steps=10,
                           report_to="none",
                           num_generations=2,
                           num_train_epochs=1,
                           temperature=1.5,
                           label_names=["labels"])

In [20]:
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[answer_accuracy_func, format_accuracy_func],
    args=training_args,
    train_dataset=dataset,
    peft_config=peft_config
)

In [21]:
trainer.train()

Step,Training Loss
10,0.002000
20,0.007000
30,-0.000000


KeyboardInterrupt: 

In [22]:
trainer.save_model('Qwen/Qwen2-0.5B-Instruct-python-sft-grpo')

In [23]:
response = []

batch_size = 2
for i in tqdm(range(0, 100, batch_size)):
    batch = dataset_test[i:i+batch_size]
    input_ids = tokenizer.batch_encode_plus(batch['prompt'], return_tensors='pt', padding=True)

    output = model.generate(
        input_ids['input_ids'].to(device), attention_mask=input_ids['attention_mask'].to(device),
        max_new_tokens=200, do_sample=False, pad_token_id=tokenizer.eos_token_id #**gen_kwargs
    )[:,input_ids['input_ids'].shape[-1]:]
    response.extend(tokenizer.batch_decode(output, skip_special_tokens=True))


  0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


In [24]:
model_ref = AutoModelForCausalLM.from_pretrained(
    'Qwen/Qwen2-0.5B-Instruct',
    torch_dtype=torch.float16,
    load_in_4bit=True
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [25]:
device = model_ref.device
response_ref = []

batch_size = 2
for i in tqdm(range(0, 100, batch_size)):
    batch = dataset_test[i:i+batch_size]
    input_ids = tokenizer.batch_encode_plus(batch['prompt'], return_tensors='pt', padding=True)

    output = model_ref.generate(
        input_ids['input_ids'].to(device), attention_mask=input_ids['attention_mask'].to(device),
        max_new_tokens=200, do_sample=False, pad_token_id=tokenizer.eos_token_id #**gen_kwargs
    )[:,input_ids['input_ids'].shape[-1]:]
    response_ref.extend(tokenizer.batch_decode(output, skip_special_tokens=True))

  0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


In [26]:
result_df = pd.DataFrame({'prompt': dataset['prompt'][:100], 'answer': dataset['answer'][:100], 'response (before)': response_ref, 'response (after)': response})

In [27]:
def parse_answer(text):
    m = re.search('```python\n(.*?)\n```', text, re.DOTALL)
    if m is not None:
        return m.group(1)
    else:
        return ""

In [28]:
result_df['answer_before'] = result_df["response (before)"].apply(parse_answer)
result_df['answer_after'] = result_df["response (after)"].apply(parse_answer)
result_df['answer_correct'] = result_df["answer"].apply(parse_answer)

In [29]:
np.mean(format_accuracy_func(result_df['response (before)']))

np.float64(0.28)

In [30]:
np.mean(answer_accuracy_func(result_df["response (before)"]))

np.float64(0.27)

In [31]:
np.mean(format_accuracy_func(result_df['response (after)']))

np.float64(0.27)

In [32]:
np.mean(answer_accuracy_func(result_df['response (after)']))

np.float64(0.26)

In [33]:
result_df.head()

,prompt,answer,response (before),response (after),answer_before,answer_after,answer_correct
0,```python\ndef scientific_to_float(scientific_...,```python\n try:\n # Use Python's bu...,```python\ndef find_missing_positive_integer(s...,```python\ndef find_missing_positive_integer(s...,,,try:\n # Use Python's built-in floa...
1,Write a function to find the count of numbers ...,```python\ndef find_smaller_than_average(lst):...,```python\nimport math\n\ndef surface_cube(edg...,```python\nimport math\n\ndef surface_cube(edg...,import math\n\ndef surface_cube(edge):\n if...,import math\n\ndef surface_cube(edge):\n if...,def find_smaller_than_average(lst):\n if no...
2,Write a function to perform a flood fill on an...,"```python\ndef flood_fill(image, sr, sc, newCo...",```python\nfrom collections import Counter\n\n...,```python\nfrom collections import Counter\n\n...,from collections import Counter\n\ndef count_w...,from collections import Counter\n\ndef count_w...,"def flood_fill(image, sr, sc, newColor):\n ..."
3,Write a function to count the number of times ...,"```python\ndef count_pattern_overlap(text, pat...","```python\ndef are_rotations(a, b):\n """"""\n...","```python\ndef are_rotations(a, b):\n """"""\n...",,,"def count_pattern_overlap(text, pattern):\n ..."
4,Write a function to convert a list of strings ...,```python\ndef to_tag_case(strings):\n resu...,```python\ndef find_max_index(arr):\n if no...,```python\ndef find_max_index(arr):\n if no...,def find_max_index(arr):\n if not arr:\n ...,def find_max_index(arr):\n if not arr:\n ...,def to_tag_case(strings):\n result = []\n ...
